In [1]:
import pandas as pd
customer = pd.read_csv("data/customer_join.csv")
uselog_months = pd.read_csv("data/use_log_months.csv")

In [2]:
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog

/var/folders/xx/_dc40rv92c551_8fcs0qtp300000gn/T/ipykernel_59981/3862046271.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.rename(columns={"count":"count_0"}, inplace=True)
/var/folders/xx/_dc40rv92c551_8fcs0qtp300000gn/T/ipykernel_59981/3862046271.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_before.rename(columns={"count":"count_1"}, inplace=True)
/var/folders/xx/_dc40rv92c551_8fcs0qtp300000gn/T/ipykernel_59981/3862046271.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

年月 customer_id  count_0  count_1
0      201805    AS002855        5      4.0
1      201805    AS009373        4      3.0
2      201805    AS015233        7      NaN
3      201805    AS015315        3      6.0
4      201805    AS015739        5      7.0
...       ...         ...      ...      ...
33846  201903    TS995853        8     11.0
33847  201903    TS998593        8      7.0
33848  201903    TS999079        3      2.0
33849  201903    TS999231        6      6.0
33850  201903    TS999855        4      4.0

[33851 rows x 4 columns]

In [3]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i]-relativedelta(months=1)
exit_customer["exit_date"] = pd.to_datetime(exit_customer["exit_date"])
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

/var/folders/xx/_dc40rv92c551_8fcs0qtp300000gn/T/ipykernel_59981/3653743321.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["exit_date"] = None
/var/folders/xx/_dc40rv92c551_8fcs0qtp300000gn/T/ipykernel_59981/3653743321.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
/var/folders/xx/_dc40rv92c551_8fcs0qtp300000gn/T/ipykernel_59981/3653743321.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

33851


年月 customer_id  count_0  count_1 name class gender start_date end_date  \
0  201805    AS002855        5      4.0  NaN   NaN    NaN        NaN      NaT   
1  201805    AS009373        4      3.0  NaN   NaN    NaN        NaN      NaT   
2  201805    AS015233        7      NaN  NaN   NaN    NaN        NaN      NaT   
3  201805    AS015315        3      6.0  NaN   NaN    NaN        NaN      NaT   
4  201805    AS015739        5      7.0  NaN   NaN    NaN        NaN      NaT   

  campaign_id  ...  price campaign_name  mean median  max  min  routine_flg  \
0         NaN  ...    NaN           NaN   NaN    NaN  NaN  NaN          NaN   
1         NaN  ...    NaN           NaN   NaN    NaN  NaN  NaN          NaN   
2         NaN  ...    NaN           NaN   NaN    NaN  NaN  NaN          NaN   
3         NaN  ...    NaN           NaN   NaN    NaN  NaN  NaN          NaN   
4         NaN  ...    NaN           NaN   NaN    NaN  NaN  NaN          NaN   

   calc_date  membership_period exit_date  
0        NaN                NaN       NaT  
1        NaN                NaN       NaT  
2        NaN                NaN       NaT  
3        NaN                NaN       NaT  
4        NaN                NaN       NaT  

[5 rows x 22 columns]

In [4]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


年月 customer_id  count_0  count_1   name class gender  start_date  \
19   201805    AS055680        3      3.0  XXXXX   C01      M  2018-03-01   
57   201805    AS169823        2      3.0     XX   C01      M  2017-11-01   
110  201805    AS305860        5      3.0   XXXX   C01      M  2017-06-01   
128  201805    AS363699        5      3.0  XXXXX   C01      M  2018-02-01   
147  201805    AS417696        1      4.0     XX   C03      F  2017-09-01   

      end_date campaign_id  ...    price campaign_name      mean median  max  \
19  2018-06-30         CA1  ...  10500.0            通常  3.000000    3.0  3.0   
57  2018-06-30         CA1  ...  10500.0            通常  3.000000    3.0  4.0   
110 2018-06-30         CA1  ...  10500.0            通常  3.333333    3.0  5.0   
128 2018-06-30         CA1  ...  10500.0            通常  3.333333    3.0  5.0   
147 2018-06-30         CA1  ...   6000.0            通常  2.000000    1.0  4.0   

     min  routine_flg   calc_date  membership_period  exit_date  
19   3.0          0.0  2018-06-30                3.0 2018-05-30  
57   2.0          1.0  2018-06-30                7.0 2018-05-30  
110  2.0          0.0  2018-06-30               12.0 2018-05-30  
128  2.0          0.0  2018-06-30                4.0 2018-05-30  
147  1.0          0.0  2018-06-30                9.0 2018-05-30  

[5 rows x 22 columns]

In [5]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [6]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


年月 customer_id  count_0  count_1   name class gender  start_date  \
0  201903    AS182308       10      7.0    XXX   C03      M  2018-12-10   
1  201808    IK733404        6      5.0   XXXX   C03      M  2016-05-01   
2  201902    HD188491        5      5.0   XXXX   C03      M  2017-01-01   
3  201805    AS621552        3      4.0  XXXXX   C02      F  2016-03-01   
4  201810    HI067846        6     11.0   XXXX   C02      F  2018-07-08   

  end_date campaign_id  ...  class_name   price  campaign_name      mean  \
0      NaN         CA3  ...         ナイト  6000.0          入会費無料  9.000000   
1      NaN         CA1  ...         ナイト  6000.0             通常  4.083333   
2      NaN         CA1  ...         ナイト  6000.0             通常  5.333333   
3      NaN         CA1  ...       デイタイム  7500.0             通常  5.250000   
4      NaN         CA1  ...       デイタイム  7500.0             通常  8.111111   

   median   max  min  routine_flg   calc_date membership_period  
0     9.5  10.0  7.0          1.0  2019-04-30               4.0  
1     4.0   6.0  1.0          1.0  2019-04-30              35.0  
2     5.0   7.0  3.0          1.0  2019-04-30              27.0  
3     5.5   8.0  2.0          1.0  2019-04-30              37.0  
4     7.0  11.0  6.0          1.0  2019-04-30               9.0  

[5 rows x 21 columns]

In [7]:
predict_data =  pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


/var/folders/xx/_dc40rv92c551_8fcs0qtp300000gn/T/ipykernel_59981/4063448895.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predict_data =  pd.concat([conti_uselog, exit_uselog], ignore_index=True)


年月 customer_id  count_0  count_1   name class gender  start_date  \
0  201903    AS182308       10      7.0    XXX   C03      M  2018-12-10   
1  201808    IK733404        6      5.0   XXXX   C03      M  2016-05-01   
2  201902    HD188491        5      5.0   XXXX   C03      M  2017-01-01   
3  201805    AS621552        3      4.0  XXXXX   C02      F  2016-03-01   
4  201810    HI067846        6     11.0   XXXX   C02      F  2018-07-08   

  end_date campaign_id  ...   price campaign_name      mean median   max  min  \
0      NaT         CA3  ...  6000.0         入会費無料  9.000000    9.5  10.0  7.0   
1      NaT         CA1  ...  6000.0            通常  4.083333    4.0   6.0  1.0   
2      NaT         CA1  ...  6000.0            通常  5.333333    5.0   7.0  3.0   
3      NaT         CA1  ...  7500.0            通常  5.250000    5.5   8.0  2.0   
4      NaT         CA1  ...  7500.0            通常  8.111111    7.0  11.0  6.0   

   routine_flg   calc_date  membership_period exit_date  
0          1.0  2019-04-30                4.0       NaT  
1          1.0  2019-04-30               35.0       NaT  
2          1.0  2019-04-30               27.0       NaT  
3          1.0  2019-04-30               37.0       NaT  
4          1.0  2019-04-30                9.0       NaT  

[5 rows x 22 columns]

In [8]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

/var/folders/xx/_dc40rv92c551_8fcs0qtp300000gn/T/ipykernel_59981/3694169734.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data["period"][i] = int(delta.years*12 + delta.months)
/var/folders/xx/_dc40rv92c551_8fcs0qtp300000gn/T/ipykernel_59981/3694169734.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data["period"][i] = int(delta.years*12 + delta.months)
/var/folders/xx/_dc40rv92c551_8fcs0qtp300000gn/T/ipykernel_59981/3694169734.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

年月 customer_id  count_0  count_1   name class gender start_date  \
0  201903    AS182308       10      7.0    XXX   C03      M 2018-12-10   
1  201808    IK733404        6      5.0   XXXX   C03      M 2016-05-01   
2  201902    HD188491        5      5.0   XXXX   C03      M 2017-01-01   
3  201805    AS621552        3      4.0  XXXXX   C02      F 2016-03-01   
4  201810    HI067846        6     11.0   XXXX   C02      F 2018-07-08   

  end_date campaign_id  ...      mean median   max  min  routine_flg  \
0      NaT         CA3  ...  9.000000    9.5  10.0  7.0          1.0   
1      NaT         CA1  ...  4.083333    4.0   6.0  1.0          1.0   
2      NaT         CA1  ...  5.333333    5.0   7.0  3.0          1.0   
3      NaT         CA1  ...  5.250000    5.5   8.0  2.0          1.0   
4      NaT         CA1  ...  8.111111    7.0  11.0  6.0          1.0   

    calc_date  membership_period  exit_date  period   now_date  
0  2019-04-30                4.0        NaT       2 2019-03-01  
1  2019-04-30               35.0        NaT      27 2018-08-01  
2  2019-04-30               27.0        NaT      25 2019-02-01  
3  2019-04-30               37.0        NaT      26 2018-05-01  
4  2019-04-30                9.0        NaT       2 2018-10-01  

[5 rows x 24 columns]

In [9]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               253
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [10]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2641
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2641
period                  0
now_date                0
dtype: int64

In [11]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

campaign_name class_name gender  count_1  routine_flg  period  is_deleted
0         入会費無料        ナイト      M      7.0          1.0       2         0.0
1            通常        ナイト      M      5.0          1.0      27         0.0
2            通常        ナイト      M      5.0          1.0      25         0.0
3            通常      デイタイム      F      4.0          1.0      26         0.0
4            通常      デイタイム      F     11.0          1.0       2         0.0

In [12]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

count_1  routine_flg  period  is_deleted  campaign_name_入会費半額  \
0      7.0          1.0       2         0.0                False   
1      5.0          1.0      27         0.0                False   
2      5.0          1.0      25         0.0                False   
3      4.0          1.0      26         0.0                False   
4     11.0          1.0       2         0.0                False   

   campaign_name_入会費無料  campaign_name_通常  class_name_オールタイム  class_name_デイタイム  \
0                 True             False              False             False   
1                False              True              False             False   
2                False              True              False             False   
3                False              True              False              True   
4                False              True              False              True   

   class_name_ナイト  gender_F  gender_M  
0            True     False      True  
1            True     False      True  
2            True     False      True  
3           False      True     False  
4           False      True     False

In [13]:
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

count_1  routine_flg  period  is_deleted  campaign_name_入会費半額  \
0      7.0          1.0       2         0.0                False   
1      5.0          1.0      27         0.0                False   
2      5.0          1.0      25         0.0                False   
3      4.0          1.0      26         0.0                False   
4     11.0          1.0       2         0.0                False   

   campaign_name_入会費無料  class_name_オールタイム  class_name_デイタイム  gender_F  
0                 True              False             False     False  
1                False              False             False     False  
2                False              False             False     False  
3                False              False              True      True  
4                False              False              True      True